In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [43]:
waitlist1 = pd.read_csv("data/Waitlist___Organ_by_Age,_Ethnicity.csv")
waitlist1.dropna(how='all', inplace=True)
waitlist1.fillna(method='ffill', inplace=True)
waitlist1.replace(",", "", regex=True, inplace=True)
for col in waitlist1.columns[2:]:
    waitlist1[col] = waitlist1[col].astype("float") 

In [78]:
def recursive_l_diverse(group):
    sorted_group = group.iloc[0, 3:].sort_values(ascending=False)
    if np.sum(sorted_group[1:]) == 0:
        return np.inf
    else:
        return sorted_group[0] / np.sum(sorted_group[1:])
    #display(group.iloc[0, 3:].sort_values(ascending=False))

    
def distinct_l_diverse(group):
    return np.sum(group.iloc[0, 3:] > 0)


def entropy_l_diverse(group):
    qblock = group.iloc[0, 3:]
    p = qblock / qblock.sum()
    return - np.sum(p * np.log(p))

In [92]:
# waitlist positive disclosure
result = pd.DataFrame(waitlist1.groupby(by=['Waitlist Age', 'Waitlist Ethnicity']) \
                      .apply(distinct_l_diverse)).reset_index()

result.columns = ["Waitlist Age", "Waitlist Ethnicity", "Distinct L Diversity"]
result = result[result["Distinct L Diversity"] == 1]

In [95]:
for index, row in result.iterrows():
    mask1 = waitlist1["Waitlist Age"] == row[0]
    mask2 = waitlist1["Waitlist Ethnicity"] == row[1]
    query = waitlist1[mask1 & mask2]

In [86]:
result1 = pd.DataFrame(waitlist1.groupby(by=['Waitlist Age', 'Waitlist Ethnicity']) \
                      .apply(entropy_l_diverse)).reset_index()

result1.columns = ["Waitlist Age", "Waitlist Ethnicity", "Entropy L Diversity"]
result1[result1["Entropy L Diversity"] <= 0.3]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log


,Waitlist Age,Waitlist Ethnicity,Entropy L Diversity
6,1-5 Years,Pacific Islander,-0.000000
30,35-49 Years,Pacific Islander,0.285792
38,50-64 Years,Pacific Islander,0.244870
41,6-10 Years,American Indian/Alaska Native,-0.000000
51,65,Black,0.269891
54,65,Pacific Islander,0.234638
